In [52]:
from operator import neg
import pandas as pd
import pickle
import requests
from requests.structures import CaseInsensitiveDict
import json
import shutil
import time

In [53]:
client_id = ""
client_secret = ""

In [54]:
# Get a bearer token.
petfinder_token_url = "https://api.petfinder.com/v2/oauth2/token"

data = {'grant_type' : 'client_credentials', 'client_id' : client_id , 'client_secret' : client_secret}

response = requests.post(petfinder_token_url, data = data)

json_response = json.loads(response.content)

bearer_token = json_response['access_token']

In [55]:
# Get Best Friends Society Organization.
petfinder_get_organizations_url = "https://api.petfinder.com/v2/organizations"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

params = {'name' : 'Best Friends Animal Society'}

response = requests.get(petfinder_get_organizations_url, params = params, headers = headers)

json_response = json.loads(response.content)

for i in range(5):
    print(json_response['organizations'][i])

{'id': 'UT81', 'name': 'Best Friends Animal Society-Utah', 'email': 'utahadoptions@bestfriends.org', 'phone': '(801) 574-2454', 'address': {'address1': '2005 S 1100 E', 'address2': None, 'city': 'Salt Lake City', 'state': 'UT', 'postcode': '84106', 'country': 'US'}, 'hours': {'monday': '10 am - 7 pm', 'tuesday': '10 am - 7 pm', 'wednesday': '10 am - 7 pm', 'thursday': '10 am - 7 pm', 'friday': '10 am - 7 pm', 'saturday': '10 am - 7 pm', 'sunday': '10 am - 7 pm'}, 'url': 'https://www.petfinder.com/member/us/ut/salt-lake-city/best-friends-animal-society-utah-ut81/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'website': 'http://utah.bestfriends.org', 'mission_statement': 'The mission of Best Friends Animal Society is to bring about a time when there are No More Homeless Pets....', 'adoption': {'policy': 'Due to Covid-19, our SLC Lifesaving Center is temporarily closed. All pets are currently in foster homes and adoption meet...', 'url': 'https://utah.bestfriends.org/get-involved/ado

In [56]:
# The Best Friends Animal Society's main office is the one with id "UT77".
# Getting 100 profiles from adoptable animals from that organization.
petfinder_get_animals_url = "https://api.petfinder.com/v2/animals"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

organization_id = "UT77"
limit = 100

params = {'organization' : organization_id, 'limit' : limit}

response = requests.get(petfinder_get_animals_url, params = params, headers = headers)

json_response = json.loads(response.content)

print(json_response)

{'animals': [{'id': 59027331, 'organization_id': 'UT77', 'url': 'https://www.petfinder.com/dog/daffy-59027331/ut/kanab/best-friends-animal-society-ut77/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'type': 'Dog', 'species': 'Dog', 'breeds': {'primary': 'Maltese', 'secondary': 'Yorkshire Terrier', 'mixed': True, 'unknown': False}, 'colors': {'primary': 'White / Cream', 'secondary': None, 'tertiary': None}, 'age': 'Adult', 'gender': 'Female', 'size': 'Small', 'coat': None, 'attributes': {'spayed_neutered': False, 'house_trained': False, 'declawed': None, 'special_needs': False, 'shots_current': True}, 'environment': {'children': None, 'dogs': None, 'cats': None}, 'tags': [], 'name': 'Daffy', 'description': None, 'organization_animal_id': 'BKNB-A-42607', 'photos': [{'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/59027331/1/?bust=1669958063&width=100', 'medium': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/59027331/1/?bust=1669958063&width=300', 'large': 'https://dl5

In [57]:
print(len(json_response['animals']))

100


In [58]:
# Harmonize data.
animals = json_response['animals']
harmonized_animals = []

for animal in animals:

    harmonized_animal = {'Type' : animal['type'], 'Name' : animal['name'], 'Age' : animal['age'],\
                           'Breed1' : animal['breeds']['primary'], 'Breed2' : animal['breeds']['secondary'], \
                           'Gender' : animal['gender'], 'Color1' : animal['colors']['primary'], \
                           'Color2' : animal['colors']['secondary'], 'Color3' : animal['colors']['tertiary'], \
                           'MaturitySize' : animal['size'], 'FurLength' : None, 'Vaccinated' : animal['attributes']['shots_current'], \
                           'Dewormed' : None, 'Sterilized' : animal['attributes']['spayed_neutered'], 'Health' : None, \
                           'Quantity' : 1, 'Fee' : None, 'State' : animal['contact']['address']['state'], 'RescuerID' : None, \
                           'VideoAmt' : len(animal['videos']), 'Description' : animal['description'], 'PetID' : animal['id'], \
                           'PhotoAmt' : len(animal['photos']), 'AdoptionSpeed' : None, 'id' : animal['id'] }
    harmonized_animals.append(harmonized_animal)

print(harmonized_animals)

[{'Type': 'Dog', 'Name': 'Daffy', 'Age': 'Adult', 'Breed1': 'Maltese', 'Breed2': 'Yorkshire Terrier', 'Gender': 'Female', 'Color1': 'White / Cream', 'Color2': None, 'Color3': None, 'MaturitySize': 'Small', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': False, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': None, 'PetID': 59027331, 'PhotoAmt': 2, 'AdoptionSpeed': None, 'id': 59027331}, {'Type': 'Dog', 'Name': 'Bugs', 'Age': 'Senior', 'Breed1': 'Chihuahua', 'Breed2': None, 'Gender': 'Male', 'Color1': 'Black', 'Color2': None, 'Color3': None, 'MaturitySize': 'Small', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': True, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': None, 'PetID': 59027330, 'PhotoAmt': 1, 'AdoptionSpeed': None, 'id': 59027330}, {'Type': 'Dog', 'Name': 'Pepe', 'Age': 'Senior', 'Breed1': 'Shih Tzu', 'Breed2': 

In [59]:
df = pd.DataFrame(harmonized_animals)

In [60]:
df.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,id
0,Dog,Daffy,Adult,Maltese,Yorkshire Terrier,Female,White / Cream,None,None,Small,...,1,None,UT,None,0,None,59027331,2,None,59027331
1,Dog,Bugs,Senior,Chihuahua,None,Male,Black,None,None,Small,...,1,None,UT,None,0,None,59027330,1,None,59027330
2,Dog,Pepe,Senior,Shih Tzu,None,Male,Gray / Blue / Silver,None,None,Small,...,1,None,UT,None,0,None,59027030,1,None,59027030
3,Cat,Herbst,Baby,Domestic Medium Hair,None,Male,White,Gray / Blue / Silver,None,Small,...,1,None,UT,None,0,"Hello, I am Herbst. I was born with a stubby t...",59023170,1,None,59023170
4,Cat,Gilaani,Baby,Domestic Medium Hair,None,Male,Gray / Blue / Silver,White,None,Small,...,1,None,UT,None,0,"Hello, my name is Gilaani. People tell me that...",59023171,1,None,59023171
5,Cat,Kanf,Baby,Domestic Medium Hair,None,Male,White,Brown / Chocolate,None,Small,...,1,None,UT,None,0,"Hello, my name is Kanf. I love to warm up my s...",59023169,1,None,59023169
6,Dog,Cambrie,Adult,Pit Bull Terrier,None,Female,Yellow / Tan / Blond / Fawn,None,None,Medium,...,1,None,UT,None,0,Cambrie is a happy go-lucky pup that can&amp;#...,59020797,3,None,59020797
7,Dog,Malekith,Young,German Shepherd Dog,Border Collie,Male,Black,None,None,Medium,...,1,None,UT,None,0,None,59018114,6,None,59018114
8,Cat,Biff,Adult,Domestic Medium Hair,None,Male,Brown / Chocolate,White,None,Small,...,1,None,UT,None,0,None,59008716,3,None,59008716
9,Cat,Jannie,Baby,Domestic Short Hair,None,Male,Orange / Red,White,None,Small,...,1,None,UT,None,0,Are you looking to up your daily dose of laugh...,59008074,1,None,59008074


In [61]:
#df.to_csv('data/best_friends_animal_society/test.csv', index = False)

In [63]:
# Applying harmonization logic.
from petfinder_animal import petfinder_animal
fourth_animal = json_response['animals'][6]

harmonized_animal = petfinder_animal.create_harmonized_petfinder_animal(fourth_animal)

# Converting harmonized animal to dataframe.
print(harmonized_animal.as_kaggle_dictionary())
harmonized_animal_df = pd.DataFrame([harmonized_animal.as_kaggle_dictionary()])
harmonized_animal_df

{'Type': 1, 'Breed1': 0, 'Breed2': 0, 'Gender': 2, 'Color1': 0, 'Color2': 0, 'Color3': 0, 'MaturitySize': 2, 'FurLength': 0, 'Vaccinated': 1, 'Dewormed': 3, 'Sterilized': 1, 'Health': 0, 'State': 0, 'HasDescription': True, 'Age': 0, 'Quantity': 1, 'Fee': 0, 'VideoAmt': 0, 'PhotoAmt': 3, 'DescriptionLength': 112, 'nltk_negative_prob': 0.0, 'nltk_neutral_prob': 0.708, 'nltk_positive_prob': 0.292, 'nltk_compound_score': 0.7906, 'AvgLaPlacianVariance': 65.84562956335752}


,Type,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,...,Quantity,Fee,VideoAmt,PhotoAmt,DescriptionLength,nltk_negative_prob,nltk_neutral_prob,nltk_positive_prob,nltk_compound_score,AvgLaPlacianVariance
0,1,0,0,2,0,0,0,2,0,1,...,1,0,0,3,112,0.0,0.708,0.292,0.7906,65.84563


In [64]:
# Load Model with pickle.
model = pickle.load(open('models/random_forest_classifier_kaggle.pkl','rb'))

In [65]:
# Call Prediction on dataframe.
print(model.predict(harmonized_animal_df))

[3]


C:\Users\oswal\anaconda3\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [50]:
print(harmonized_animal.Description)

Hello, my name is Kanf. I love to warm up my singing voice in the mornings while you are preparing...


In [51]:
# Getting feedback from the animal:
from petfinder_feedback import petfinder_feedback

feedback = petfinder_feedback(harmonized_animal)
print(feedback.sentence_sentiments)
print(feedback.photo_blurriness)

{'Hello, my name is Kanf.': 0.0, 'I love to warm up my singing voice in the mornings while you are preparing...': 0.0}
{'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/59023169/1/?bust=1669996646': 66.31807747124567}


In [ ]:
# Check for other animals.
for animal in json_response['animals']:
    harmonized_animal = petfinder_animal.create_harmonized_petfinder_animal(animal)
    print("Feedback ")
    feedback = petfinder_feedback(harmonized_animal)
    print(feedback.sentence_sentiments)
    #print(feedback.photo_blurriness)

In [ ]:
#harmonized_animal.Photos[0]['small']
#res = requests.get(harmonized_animal.Photos[0]['small'], stream = True)
#with open("temp.jpg",'wb') as f:
#        shutil.copyfileobj(res.raw, f)
#urllib.urlretrieve(harmonized_animal.Photos[0]['small'], full_path)